## What this notebook is
This notebook demonstrates how I trained Gemma-2 9b to obtain LB: 0.941. The inference code can be found [here](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora).
I used 4-bit quantized [Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit) uploaded by unsloth team as a base-model and added LoRA adapters and trained for 1 epoch.

## Result

I used `id % 5 == 0` as an evaluation set and used all the rest for training.

| subset | log loss |
| - | - |
| eval | 0.9371|
| LB | 0.941 |

## What is QLoRA fine-tuning?

In the conventional fine-tuning, weight ($\mathbf{W}$) is updated as follows:

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

where $L$ is a loss at this step and $\eta$ is a learning rate.

[LoRA](https://arxiv.org/abs/2106.09685) tries to approximate the $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ by factorizing $\Delta \mathbf{W}$ into two (much) smaller matrices, $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ and $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ with $r \ll \text{min}(\text{d}, \text{k})$.

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

During training, only $\mathbf{A}$ and $\mathbf{B}$ are updated while freezing the original weights, meaning that only a fraction (e.g. <1%) of the original weights need to be updated during training. This way, we can reduce the GPU memory usage significantly during training while achieving equivalent performance to the usual (full) fine-tuning.

[QLoRA](https://arxiv.org/abs/2305.14314) pushes the efficiency further by quantizing LLM. For example, a 8B parameter model alone would take up 32GB of VRAM in 32-bit, whereas quantized 8-bit/4-bit 8B model only need 8GB/4GB respectively. 
Note that QLoRA only quantize LLM's weights in low precision (e.g. 8-bit) while the computation of forward/backward are done in higher precision (e.g. 16-bit) and LoRA adapter's weights are also kept in higher precision.

1 epoch using A6000 took ~15h in 4-bit while 8-bit took ~24h and the difference in log loss was not significant.

## Note
It takes prohivitively long time to run full training on kaggle kernel. I recommend to use external compute resource to run the full training.
This notebook uses only 100 samples for demo purpose, but everything else is same as my setup.

In [1]:
# gemma-2 is available from transformers>=4.42.3
# !pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [1]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

c:\Users\asdzx\anaconda3\envs\yolov7\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configurations

In [2]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # global batch size is 8 
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

#### Training Arguments

In [3]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

#### LoRA config

In [4]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

### Instantiate the tokenizer & model

In [5]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

In [6]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0): Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
              (rotary_emb): Gemma2RotaryEmbedding()
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)

In [7]:
model.print_trainable_parameters()

trainable params: 7,891,456 || all params: 9,249,608,192 || trainable%: 0.0853


### Instantiate the dataset

In [8]:
ds = Dataset.from_csv("data/train.csv")

In [9]:
import json

class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        try:
            # 用 json.loads 解析 JSON 格式的字串
            parsed_list = json.loads(text)
            return " ".join(parsed_list)
        except (json.JSONDecodeError, TypeError) as e:
            print(f"Error processing text: {text}. Error: {e}")
            return text

In [11]:
test_batch = {
    "id":["1", "2"],
    "prompt": ["'你好'", "'天氣怎麼樣'"],
    "response_a": ["'很好'", "'晴天'"],
    "response_b": ["'還行'", "'多雲'"],
    "winner_model_a": [True, False],
    "winner_model_b": [False, True],
}
custom_tokenizer = CustomTokenizer(tokenizer, max_length=1024)
output = custom_tokenizer(test_batch)
print("Tokenized input_ids:", output["input_ids"])
print("Labels:", output["labels"])


Error processing text: '你好'. Error: Expecting value: line 1 column 1 (char 0)
Error processing text: '天氣怎麼樣'. Error: Expecting value: line 1 column 1 (char 0)
Error processing text: '很好'. Error: Expecting value: line 1 column 1 (char 0)
Error processing text: '晴天'. Error: Expecting value: line 1 column 1 (char 0)
Error processing text: '還行'. Error: Expecting value: line 1 column 1 (char 0)
Error processing text: '多雲'. Error: Expecting value: line 1 column 1 (char 0)
Tokenized input_ids: [[2, 235322, 39038, 78880, 777, 87139, 235303, 109, 235322, 4250, 235298, 235250, 78880, 777, 40487, 235303, 109, 235322, 4250, 235298, 235268, 78880, 777, 236570, 235599, 235303, 1], [2, 235322, 39038, 78880, 777, 203226, 44037, 236983, 235303, 109, 235322, 4250, 235298, 235250, 78880, 777, 237736, 235654, 235303, 109, 235322, 4250, 235298, 235268, 78880, 777, 235626, 237887, 235303, 1]]
Labels: [0, 1]


In [12]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/57477 [00:00<?, ? examples/s]

Error processing text: ["I do not have any confirmed details about a World War III. Speculation about future conflicts can often be misleading or anxiety-provoking. Most experts believe maintaining open communication and cooperation between countries is the best way to prevent large-scale wars.",null]. Error: sequence item 1: expected str instance, NoneType found
Error processing text: ["Stochastic Gradient Langevin Dynamics (SGLD) is an algorithm used for Bayesian learning and inference in machine learning. It combines the goodness of Stochastic Gradient Descent (SGD) and Langevin dynamics to perform parameter estimation. Here's a simplified explanation of how it works:\n\n1. Initialization: Start with an initial point (parameters) randomly.\n\n2. Mini-batch Gradient Computation: Compute the gradient of the loss function with respect to the parameters using a small subset of the data (a mini-batch). This is similar to how SGD works.\n\n3. Update with Noise: Update the parameters by ta

Map:   2%|▏         | 1000/57477 [00:00<00:23, 2451.22 examples/s]

Error processing text: ["Sure, I'd be happy to help you create a game! Can you give me a little more information about what you have in mind? Here are a few questions to get started:\n\n1. What is the genre of the game? (e.g. action, adventure, puzzle, strategy, etc.)\n2. What is the setting of the game? (e.g. fantasy world, sci-fi universe, historical period, etc.)\n3. Who is the target audience for the game? (e.g. children, teenagers, adults, etc.)\n4. What is the main objective of the game? (e.g. complete a series of levels, defeat an enemy, collect a certain number of items, etc.)\n\nOnce you provide me with some more information, I can start generating ideas for your game!","To create a list of words in a specific order, you can use the `sorted()` function in Python. Here's an example:\n```\nwords = [\"apple\", \"banana\", \"cherry\"]\n\nsorted_words = sorted(words)\n\nprint(sorted_words)\n```\nThis will output the list of words in alphabetical order:\n```\n['apple', 'banana', 'ch

Map:   3%|▎         | 2000/57477 [00:00<00:22, 2465.63 examples/s]

...

Map:  96%|█████████▌| 55000/57477 [00:24<00:01, 2161.66 examples/s]

Error processing text: [null,"The correct statement about the program CUDA threads must execute is:\n\nAll threads inside a thread block must execute the same non-branching sequential program but different thread blocks can have different programs.","In CUDA, a thread block is a group of threads that execute the same program concurrently on a single GPU stream processor. Each thread within a thread block executes a separate instance of the program on different data. In order for the program to be executed correctly, it's important that all threads within a thread block execute the same non-branching sequential program.\n\nA non-branching sequential program is a program that does not contain any branching statements (e.g. `if` statements, `else` statements, `while` statements) that allow execution to be redirected to different parts of the program. This is because if a branching statement were present, it could cause some threads to execute different parts of the program, which would re

Map:  97%|█████████▋| 56000/57477 [00:24<00:00, 2136.14 examples/s]

Error processing text: ["The Polish death camps were a collective term used by the Nazi regime to refer to concentration camps in Poland where Jews, Gypsies, and other non-Jewish individuals were held for various reasons, including the persecution of the Jewish people by the Nazis during World War II. These camps were designed specifically for the extermination of Jews and other non-Jews, and were operated by the German government during the Holocaust.\nSome of the most well-known Polish death camps include:\n1. Ghettos in Katyn and other northern Poland towns\n2. Auschwitz concentration camp in Poland\n3. Buchenwald, Germany\n4. Birkenau concentration camp in Poland\n5. Treblinka concentration camp in Poland\n6. Auschwitz concentration camp in Poland\n7. Birkenau concentration camp in Poland\n8. Buchenwald concentration camp in Germany\n9. Auschwitz concentration camp in Poland\n10. Auschwitz concentration camp in Poland\nThe Holocaust was a global genocide that took place during Worl

Map: 100%|██████████| 57477/57477 [00:25<00:00, 2241.74 examples/s]


### Compute metrics

We'll compute the log-loss used in LB and accuracy as a auxiliary metric.

In [13]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

### Split

Here, train and eval is splitted according to their `id % 5`

In [14]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

C:\Users\asdzx\AppData\Local\Temp\ipykernel_12136\2739703425.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 1/11495 [00:07<24:59:09,  7.83s/it]